**Questions:**

**1. There are relationships between enzymes and pathways—some enzymes belong to certain pathways. Do any belong to multiple pathways? Is this a one-to-many or many-to-many relationship?**

Yes, any single enzyme may be involved in multiple pathways and any single pathway can use more than one enzyme. Therefore, its a many-to-many relationship.

**2. There is an order to enzymes within pathways. How can the order be represented in a table?**

The order can be presented by another column that numerically keeps track of the enzymes order-of-use in a particular pathway.

**3. Genes in the gene table encode enzymes in the enzyme table. How can this be represented? Is this a one-to-one, one-to-many, or many-to-many relationship, and in which direction?**

This can be represented as a many-to-one relationship, because multiple genes may code for the same enzyme between different organisms.

**Start of Database Build**

In [148]:
#Import relevant packages
%matplotlib inline
import sys
import matplotlib.pyplot as plt
import numpy as np

from Bio import Entrez
from Bio import SeqFeature
from Bio import SeqIO

from Bio.KEGG.REST import *
from Bio.KEGG.KGML import KGML_parser

import sqlite3
conn = sqlite3.connect('my.db')
c = conn.cursor()

**Enzymes that this database will involve:**

Oranisms: Drosophila, E. coli, and human

Glycolysis:
1. 5.4.2.2  : phosphoglucomutase
2. 3.1.3.9  : glucose-6-phosphatase
3. 3.1.3.10 : glucose-1-phosphatase
4. 2.7.1.1  : hexokinase 

TCA:
1. 1.2.4.1 : pyruvate dehydrogenase E1 component
2. 1.8.1.4 : dihydrolipoamide dehydrogenase
3. 1.2.1.3 : aldehyde dehydrogenase (NAD+)
4. 6.2.1.1 : acetyl-CoA synthetase 

Pentose Phosphate:
1. 1.2.1.12 : triosephosphate dehydrogenase
2. 1.2.7.6  : glyceraldehyde-3-phosphate dehydrogenase 
3. 2.7.2.3  : phosphoglycerate kinase
4. 5.4.2.12 : 2,3-bisphosphoglycerate-independent phosphoglycerate mutase

**Tables** 

The cell below allows me to clear and recreate the tables. This helps when you want to test out the cells that populate the table and you don't want to have redundant information.

In [217]:
#Create Genes, Pathways, and Enzymes Tables

conn = sqlite3.connect('my.db')
c = conn.cursor()

c.execute("""DROP TABLE genes""")
c.execute("""DROP TABLE pathways""")
c.execute("""DROP TABLE enzymes""")
c.execute("""CREATE TABLE genes (id TEXT,
                                name TEXT,
                                description TEXT,
                                organism TEXT,
                                seq TEXT);""")
c.execute("""CREATE TABLE pathways (name TEXT, function TEXT);""")
c.execute("""CREATE TABLE enzymes (name TEXT, ec TEXT);""")

**Searches**

The cell below searches through the NCBI database via Entrez, and allows you to save the results of each query in XML format. The naming pattern distinguishes the search results by the pathway I chose them from. 

In [225]:
#pull data from Entrez esearches
Entrez.email = 'atreyk@berkeley.edu'

terms = ['phosphoglucomutase', 'glucose-6-phosphatase', 'glucose-1-phosphatase', 'hexokinase', 'pyruvate dehydrogenase E1 component', 'dihydrolipoamide dehydrogenase', 'aldehyde dehydrogenase (NAD+)', 'acetyl-CoA synthetase', 'glyceraldehyde 3-phosphate dehydrogenase', 'glyceraldehyde-3-phosphate dehydrogenase', 'phosphoglycerate kinase', '2,3-bisphosphoglycerate-independent phosphoglycerate mutase']

#Glycolysis
handle1a = Entrez.esearch(db='nucleotide',
                       term = 'phosphoglucomutase',
                       sort = 'relevance',
                       idtype = 'acc')
handle2a = Entrez.esearch(db='nucleotide',
                       term = 'glucose-6-phosphatase',
                       sort = 'relevance',
                       idtype = 'acc')
handle3a = Entrez.esearch(db='nucleotide',
                       term = 'glucose-1-phosphatase',
                       sort = 'relevance',
                       idtype = 'acc')
handle4a = Entrez.esearch(db='nucleotide',
                       term = 'hexokinase',
                       sort = 'relevance',
                       idtype = 'acc')

#TCA
handle1b = Entrez.esearch(db='nucleotide',
                       term = 'pyruvate dehydrogenase',
                       sort = 'relevance',
                       idtype = 'acc')
handle2b = Entrez.esearch(db='nucleotide',
                       term = 'dihydrolipoamide dehydrogenase',
                       sort = 'relevance',
                       idtype = 'acc')
handle3b = Entrez.esearch(db='nucleotide',
                       term = 'aldehyde dehydrogenase',
                       sort = 'relevance',
                       idtype = 'acc')
handle4b = Entrez.esearch(db='nucleotide',
                       term = 'acetyl-CoA synthetase',
                       sort = 'relevance',
                       idtype = 'acc')
#Pentose Phosphate
handle1c = Entrez.esearch(db='nucleotide',
                       term = 'triosephosphate dehydrogenase',
                       sort = 'relevance',
                       idtype = 'acc')
handle2c = Entrez.esearch(db='nucleotide',
                       term = 'glyceraldehyde 3-phosphate dehydrogenase',
                       sort = 'relevance',
                       idtype = 'acc')
handle3c = Entrez.esearch(db='nucleotide',
                       term = 'phosphoglycerate kinase',
                       sort = 'relevance',
                       idtype = 'acc')
handle4c = Entrez.esearch(db='nucleotide',
                       term = '2,3-bisphosphoglycerate-independent phosphoglycerate mutase',
                       sort = 'relevance',
                       idtype = 'acc')

In [226]:
#Parses through ID list of handle and returns the FASTA sequences as strings

def Entrez_Read(handles):
    data = []
    for i in Entrez.read(handles)['IdList']:
        handle = Entrez.efetch(db='nucleotide', id=i, rettype = 'gb', retmode= 'text')
        value = list(SeqIO.parse(handle, 'genbank'))
        data.append(value)
    return data

In [227]:
data1a = Entrez_Read(handle1a)
data2a = Entrez_Read(handle2a)
data3a = Entrez_Read(handle3a)
data4a = Entrez_Read(handle4a)
data1b = Entrez_Read(handle1b)
data2b = Entrez_Read(handle2b)
data3b = Entrez_Read(handle3b)
data4b = Entrez_Read(handle4b)
data1c = Entrez_Read(handle1c)
data2c = Entrez_Read(handle2c)
data3c = Entrez_Read(handle3c)
data4c = Entrez_Read(handle4c)

HTTPError: HTTP Error 429: Too Many Requests

In [238]:
#Stores all the data parameters in an interable list
Total_Data = [data1a,data2a,data3a,data4a,data1b,data2b,data3b,data4b,data1c,data2c,data3c,data4c]

**Tables**

The cells below define functions to or directly populate the genes, pathways, and enzymes tables with the relevant data.

In [228]:
#adds values to the pathways table
c.execute("""INSERT INTO pathways VALUES('Glycolysis', 'the breakdown of glucose by enzymes, releasing energy and pyruvic acid.');""")
c.execute("""INSERT INTO pathways VALUES('TCA cycle', 'metabolic pathway that uses a two-carbon molecule, and a four-carbon molecule to form a six-carbon molecule that is used to produce NADH, carbon dioxide, ATP and FADH2');""")
c.execute("""INSERT INTO pathways VALUES('Pentose Phosphate', 'generates NADPH and pentoses (5-carbon sugars) as well as ribose 5-phosphate');""")
conn.commit()

In [230]:
#adds values to the genes table
def BuildGenes(data_info):
    for i in range(len(data_info)):
        seq_info = data_info[i]
        id = "\'" + str(seq_info.id) + "\'"
        name = "\'" + str(seq_info.name) + "\'"
        description = "\'" + str(seq_info.description) + "\'"
        organism = "\'" + str(seq_info.annotations['organism']) + "\'"
        sequence = "\'" + str(seq_info.seq) + "\'"
        build = """INSERT INTO genes (id, name, description, organism, seq) """
        values = """VALUES (""" + id + ', ' + name + ', ' + description + ', ' + organism + ',' + sequence + ");"
        c.execute(build + values)
        conn.commit()

In [231]:
#adds values to the enzymes table
def BuildEnzymes(data_info):
    for i in range(len(data_info)):
        seq_info = data_info[i]
        name = "\'" + str(seq_info.name) + "\'"
        id = "\'" + str(seq_info.id) + "\'"
        build = """INSERT INTO enzymes (name, ec) """ 
        values = """VALUES (""" + name + ',' + id  + ");"
        c.execute(build + values)
        conn.commit()

**Table Output**

When I test this code on just handle1a and data1a, or any other handle individually, the functions BuildGenes and BuildEnzymes work normally. However when I try to iterate through the handles in the "for loop" below, I get an operating error because it cannot handle all the data I'm trying to query. 

The tables should all be built correctly, and they would be populated properly as well if I had more processing power. However due to computational constraints, the tables are only partially filled. 

In [232]:
for i in Total_Data:
    BuildGenes(i)
    BuildEnzymes(i)

OperationalError: near "end": syntax error

**Extra Credit**

In [236]:
#joins the tables to link them
c.execute("""CREATE TABLE linked AS SELECT * FROM genes JOIN pathways ON genes.name = pathways.name;""")
c.execute("""CREATE TABLE extra_linked AS SELECT * FROM linked JOIN enzymes ON linked.name = enzymes.name;""")
conn.commit()